In [1]:
import pandas as pd
from geopy.distance import lonlat, distance
import numpy as np
from pandas.tseries.offsets import DateOffset

In [2]:
git_hub = "C:\\Users\\yjpar\\Documents\\GitHub\\RGTM\\"
#apt_geocode = pd.read_csv("C:\\Users\\Young Jin Park\\Documents\\GitHub\\RGTM\\Cleaned//merged_manhattan_geocode.csv")

In [3]:
def calculate_x_mile_vals(df, thr, value_col, fn, dist_df=None, merge_df=None):
    values=[]
    
    for i in range(len(df)):
        center = df.loc[i,"GeoCode"]
        sales_date = df.loc[i, "SaleDate"]
        if dist_df is None:
            temp_df = df.copy(deep=True)
        else:
            temp_df = dist_df.copy(deep=True)
        print(i)
        temp_df = temp_df[temp_df["GeoCode"]!=center].copy(deep=True)
        temp_df["Distance"] = temp_df["GeoCode"].apply(lambda x : distance(lonlat(*center), lonlat(*x)).miles)
        print("Calculated Distance")
        temp_df = merge_df.merge(temp_df, how="inner", on ="GeoCode")
        print("Merged Df")
        temp_df[value_col] = temp_df[value_col].apply(lambda x : float(x.replace(",","")))
        print("changed value col type")
        
        if fn == "avg":
            print("getting_into_calc")
            print(len(temp_df[(temp_df["Distance"]<thr)&
                                  (temp_df["SaleDate"]>(sales_date-DateOffset(month=6)))&
                                  (temp_df["SaleDate"]<=sales_date)]))
            values.append(temp_df[(temp_df["Distance"]<thr)&
                                  (temp_df["SaleDate"]>(sales_date-DateOffset(month=6)))&
                                  (temp_df["SaleDate"]<=sales_date)][value_col].mean())
        elif fn == "count":
            values.append(temp_df[temp_df["Distance"]<thr][value_col].count())
        elif fn == "min_dist":
            values.append(temp_df["Distance"].min())
        else:
            print("fn_not defined. Please pass in avg,count or min_dist")
    
    df["Values"] = values
    
    return df
        
        
    
    

In [4]:
apt_df = pd.read_csv(git_hub + "Cleaned\\merged_with_geocode.csv")
apt_df["GeoCode"] = apt_df["GeoCode"].apply(lambda x: eval(x))
apt_df["SaleDate"] = pd.to_datetime(apt_df["SaleDate"])
distinct_apt = pd.read_csv(git_hub + "Cleaned\\merged_manhattan_geocodes.csv")
distinct_apt["GeoCode"] = distinct_apt["GeoCode"].apply(lambda x : eval(x))
merged = pd.read_csv(git_hub + "Cleaned\\FINAL_MERGED_MANHATTAN.csv")
merged["GeoCode"] = merged["GeoCode"].apply(lambda x : eval(x))
merged["SaleDate"] = pd.to_datetime(merged["SaleDate"])
merged = merged.loc[7801:,:].reset_index()

#feature_df = pd.read_csv(git_hub + "Cleaned\\nycopen_zillow_matched_data.csv") 

#distinct_apt["StreetAddress"] = distinct_apt["Address"].apply(lambda x : x.split(",")[0])
#distinct_apt["GeoCode"] = distinct_apt["GeoCode"].apply(lambda x : eval(x))
#apt_df["GeoCode"] = apt_df["GeoCode"].apply(lambda x : eval(x))

#feature_df["StreetAddress"] = feature_df.StreetNum + " " + feature_df.StreetName

#merged = feature_df.merge(distinct_apt[["StreetAddress", "GeoCode"]], how="left", on="StreetAddress")

In [5]:
merged1 = calculate_x_mile_vals(merged, thr=0.5, value_col="SalePrice", fn="avg", dist_df=distinct_apt, merge_df=apt_df)

0
Calculated Distance
Merged Df
changed value col type
getting_into_calc
85
1
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2
Calculated Distance
Merged Df
changed value col type
getting_into_calc
119
3
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
4
Calculated Distance
Merged Df
changed value col type
getting_into_calc
121
5
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
6
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
7
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
8
Calculated Distance
Merged Df
changed value col type
getting_into_calc
141
9
Calculated Distance
Merged Df
changed value col type
getting_into_calc
45
10
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
11
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
12
Calculated Distance
Merged Df
changed value col type
getting_into_calc
48
13
Calculated

0
107
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
108
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
109
Calculated Distance
Merged Df
changed value col type
getting_into_calc
250
110
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
111
Calculated Distance
Merged Df
changed value col type
getting_into_calc
360
112
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
113
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
114
Calculated Distance
Merged Df
changed value col type
getting_into_calc
374
115
Calculated Distance
Merged Df
changed value col type
getting_into_calc
522
116
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
117
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
118
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
119
Calculated Distance
Merged Df
changed value col type
getting_i

0
213
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
214
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
215
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
216
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
217
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
218
Calculated Distance
Merged Df
changed value col type
getting_into_calc
272
219
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
220
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
221
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
222
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
223
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
224
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
225
Calculated Distance
Merged Df
changed value col type
getting_into_ca

0
319
Calculated Distance
Merged Df
changed value col type
getting_into_calc
135
320
Calculated Distance
Merged Df
changed value col type
getting_into_calc
185
321
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
322
Calculated Distance
Merged Df
changed value col type
getting_into_calc
268
323
Calculated Distance
Merged Df
changed value col type
getting_into_calc
226
324
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
325
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
326
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
327
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
328
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
329
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
330
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
331
Calculated Distance
Merged Df
changed value col type
getting_i

79
425
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
426
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
427
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
428
Calculated Distance
Merged Df
changed value col type
getting_into_calc
112
429
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
430
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
431
Calculated Distance
Merged Df
changed value col type
getting_into_calc
451
432
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
433
Calculated Distance
Merged Df
changed value col type
getting_into_calc
154
434
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
435
Calculated Distance
Merged Df
changed value col type
getting_into_calc
144
436
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
437
Calculated Distance
Merged Df
changed value col type
getting_

...

2103
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2104
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2105
Calculated Distance
Merged Df
changed value col type
getting_into_calc
373
2106
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2107
Calculated Distance
Merged Df
changed value col type
getting_into_calc
297
2108
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2109
Calculated Distance
Merged Df
changed value col type
getting_into_calc
396
2110
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2111
Calculated Distance
Merged Df
changed value col type
getting_into_calc
255
2112
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2113
Calculated Distance
Merged Df
changed value col type
getting_into_calc
306
2114
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2115
Calculated Distance
Merged Df
changed value col t

Merged Df
changed value col type
getting_into_calc
0
2208
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2209
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2210
Calculated Distance
Merged Df
changed value col type
getting_into_calc
197
2211
Calculated Distance
Merged Df
changed value col type
getting_into_calc
198
2212
Calculated Distance
Merged Df
changed value col type
getting_into_calc
462
2213
Calculated Distance
Merged Df
changed value col type
getting_into_calc
170
2214
Calculated Distance
Merged Df
changed value col type
getting_into_calc
157
2215
Calculated Distance
Merged Df
changed value col type
getting_into_calc
147
2216
Calculated Distance
Merged Df
changed value col type
getting_into_calc
160
2217
Calculated Distance
Merged Df
changed value col type
getting_into_calc
135
2218
Calculated Distance
Merged Df
changed value col type
getting_into_calc
298
2219
Calculated Distance
Merged Df
changed value col type
getting_into_

Merged Df
changed value col type
getting_into_calc
0
2312
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2313
Calculated Distance
Merged Df
changed value col type
getting_into_calc
186
2314
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2315
Calculated Distance
Merged Df
changed value col type
getting_into_calc
202
2316
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2317
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2318
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2319
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2320
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2321
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2322
Calculated Distance
Merged Df
changed value col type
getting_into_calc
231
2323
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2324


2416
Calculated Distance
Merged Df
changed value col type
getting_into_calc
186
2417
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2418
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2419
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2420
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2421
Calculated Distance
Merged Df
changed value col type
getting_into_calc
370
2422
Calculated Distance
Merged Df
changed value col type
getting_into_calc
233
2423
Calculated Distance
Merged Df
changed value col type
getting_into_calc
335
2424
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2425
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2426
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2427
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2428
Calculated Distance
Merged Df
changed value col typ

0
2520
Calculated Distance
Merged Df
changed value col type
getting_into_calc
166
2521
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2522
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2523
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2524
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2525
Calculated Distance
Merged Df
changed value col type
getting_into_calc
20
2526
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2527
Calculated Distance
Merged Df
changed value col type
getting_into_calc
21
2528
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2529
Calculated Distance
Merged Df
changed value col type
getting_into_calc
16
2530
Calculated Distance
Merged Df
changed value col type
getting_into_calc
19
2531
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
2532
Calculated Distance
Merged Df
changed value col typ

In [68]:
merged.dropna().to_csv(git_hub+"Cleaned\\FINAL_MERGED_MANHATTAN.csv")

In [6]:
merged1.to_csv(git_hub+"Cleaned\\FINAL_MERGED_MANHATTAN_yj_laptop.csv")